V tomto notebooku si ukážeme, jak jednoduše načíst marcový dokument pomocí knihovny pymarc, jak marcový záznam vypadá, jak data z marcového dokumentu vytáhnout a uložit do csv tabulky. Nakonec z dat zjistíme nejčetnějšího autora za každý rok.   


### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [ ]:
# Prikaz ktery naistaluje knihovny
%pip install pandas 
%pip install pymarc 
%pip install openpyxl

# Prikaz ktery knihovny prida 
from collections import Counter
import pandas as pd
from pymarc import MARCReader

# ROZDELIT

### 3. Načtení z csv

Teď, když máme hodnoty uložené v csv, bude mnohem jednodušší s nimi pracovat. Uložená data načteme pomocí knihovny pandas do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v csv spojené středníkem. V DataFramu je pak podpole rozpojíme a převede do listu (seznamu). 


In [ ]:
# Cesta k nasim datum
csv_data = 'data/csv/out_smz.csv'

# Nacteni dat
df = pd.read_csv(csv_data, delimiter=',')

print("Data načtena do DataFramu df.")

for column in df.columns:
    # Hodnoty spojene v jeden string zpatky rozdelime do listu, aby se nam s nim lepe pracovalo
    df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])
        
df['year'] = df['year'].apply(lambda x: x[0] if len(x)>0 else None)        

### 4. Souhrnné statistiky

Nyní se podíváme, jak z dat zjistit jednoduché statistiky. 

#### 4.1 Počet výskytů 

Pokud chceme vědet, kolikrát se v databázi objevuje nějaká hodnota, použijeme funkci `value_counts()`. V následujícím příkladu si ukážeme, kolik je v databázi z jakého roku záznamů.

In [ ]:
records_to_print = 20

df['year'].value_counts()[:records_to_print] # vypise prvnich x zaznamu

Vidíme, že zdaleka nejvíce záznamů je z 80. let, následně pak z let 70. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Místo sloupce 'year' můžeme vypsat např. 'author', čímž zjistíme, kolikrát se jaký autor v databázi objevuje.
</div>

#### 4.2 Nejčastější autor v daném roce

Teď si ukážeme, jak z dat zjistit, který autor v jakém roce publikoval nejvíce článků (tedy se v záznamech v daném roce nejčastěji objevuje jako autor). Existuje několik způsobů, jak na to přijít. My si představíme dva.<br>

##### 4.2.1 První varianta

Nejprve si napíšeme dvě vlastní funkce. První nám spočte počet článků každého autora v každém roce. Druhá pak pro každý rok zjistí, který autor v každém roce napsal nejvíce článků.  


In [ ]:
# Funkce ktera nam spocte pocet clanku v danem roce pro kazdeho autora
def count_years(df, column):
    
    # Dictionary, ve kterem jako klic ulozime (rok, jmeno autora)
    # a jako hodnotu budeme pripocitavat pocet clanku
    count = {}
    
    # Iterace pres radky DataFramu
    for _,row in df.iterrows():
        year = row['year']
        for element in row[column]:
            key = (year, element) 
            # Pokud klic jiz existuje, pripocteme jednicku
            if key in count.keys():
                count[key] += 1
            else:
                # Pokud klic zatim neexistuje, vytvorime ho a pridame jedna
                count[key] = 1  
    return count              

# Funkce, ktera nam najde nejcastejsiho autora v danem roce
def find_most_common(count):
    # Dictionary, ve kterem jako klic ulozime rok
    # a jako hodnotu (jmeno autora, pocet clanku)
    most_common = {}

    # Iterace skrz dictionary
    for key,value in count.items():
        year = key[0]
        # Pokud klic (rok) jiz existuje, podivame se, 
        # zda je pocet clanku u aktualniho autora vyssi  
        if year in most_common:
            if most_common[year][1] < value:
                # Pamatujeme si jen tu nejvyssi hodnotu
                most_common[year] = (key[1], value)    
        else:
            most_common[year] = (key[1], value) 
    
    #Klice slovniku seradime od nejmensiho cisla po nejvetsi         
    years = list(most_common.keys())
    years.sort()
    sorted_most_common = {i: most_common[i] for i in years}        
    return sorted_most_common            

print("Funkce uloženy.")
  

Teď už jen funkce zavoláme a výsledek vypíšeme. <br> 

In [ ]:
# Vybereme sloupec, ktery chceme vypsat
picked_column = 'author'

count = count_years(df, picked_column)

most_common = find_most_common(count)

# Formatovani pro hezci vypis
string = "{year: >30} {name: >30} {articles: >30}"
# Nejcastejsi hodnoty vypiseme
print(string.format(year = "Rok", name = "Nejčastější autor", articles = "Počet článků"))
print("----------------------------------------------------------------------------------------------------")
for key, value in most_common.items():
    print(string.format(year = key, name = value[0], articles = value[1]))        


V tabulce vidíme, že počet napsaných článků za jednotlivé roky se výrazně liší. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kód je napsán tak, aby se parametry daly jednoduše upravit. Můžeme si tak zkusit změnit proměnnou picked_colum na 'figures', čímž zjistíme, o kom se nejvíce daný rok psalo.
</div>

#### 4.2.2 Druhá varianta

Tím, že máme data uložena do DataFramu, umožňuje nám to využít funkce knihovny pandas. V našem případě využijeme funkci , která data seskupí podle vybraného sloupce. <br>
Použijeme také funkci `Counter`, která nám spočte výskyt každého elementu. <br>
V neposlední řadě využijeme anonymní funkci lambda.<br>

In [ ]:
# Data ocistime o prazdne hodnoty
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Data seskupime podle nejcastejsiho elementu v kazde hodnote groupby
most_common = data_filtered.groupby('year')[picked_column].apply(lambda x: Counter(element for elements in x for element in elements).most_common(1)[0][0] if len(x) > 0 else None) 

print(most_common)

V tabulce můžeme vidět, který autor za daný rok publikoval nejvíce článků napříč všemi časopisy. Je třeba si uvědomit, že i když je v tabulce nějaký autor v daný rok nejčastější, nemusí v celkovém počtu napsat hodně článků. Ostatně se o tom můžeme přesvěsvědčit v předchozí buňce.<br>
Je možné, že se od sebe dvě tabulky budou mírně odlišovat a to v případě, že za daný rok napsalo více autorů stejný počet článků. 

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Kromě bibliografie literárního exilu můžeme vyzkoušet i jiné bibliografie.
</div>
